<a href="https://colab.research.google.com/github/Dritisri/NLP/blob/main/2403a52236_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
file_path = "/content/LDA-Data.xlsx"


In [5]:
import pandas as pd
df = pd.read_excel(file_path)
df

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [6]:
import pandas as pd
df = pd.read_excel(file_path)
df = df[['News']]
df

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    # Correcting 'lemmatize_word(word)' to 'lemmatizer.lemmatize(word)'
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Correcting the column name from 'Text' to 'News'
df["Processed_Text"] = df["News"].apply(preprocess_text)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,News,Processed_Text
0,Virat scored century in match,virat scored century match
1,BJP won in elections,bjp election
2,Bumra took 5 wicket in a match,bumra took wicket match
3,Congress form state government,congress form state government


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df["Processed_Text"])


In [9]:
bow_df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names_out())
bow_df


,bjp,bumra,century,congress,election,form,government,match,scored,state,took,virat,wicket
0,0,0,1,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,1,1,0,0,1,0,0,0


In [10]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=2,
    random_state=42
)

lda.fit(bow)


LatentDirichletAllocation(n_components=2, random_state=42)

In [11]:
words = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"\nTopic {topic_idx + 1}:")
    print(", ".join([words[i] for i in topic.argsort()[-5:]]))


Topic 1:
election, state, congress, government, form

Topic 2:
took, scored, century, virat, match


In [12]:
topic_values = lda.transform(bow)
df["Dominant_Topic"] = topic_values.argmax(axis=1) + 1
df

,News,Processed_Text,Dominant_Topic
0,Virat scored century in match,virat scored century match,2
1,BJP won in elections,bjp election,1
2,Bumra took 5 wicket in a match,bumra took wicket match,2
3,Congress form state government,congress form state government,1


In [13]:
import pandas as pd
d=pd.read_csv('/content/arxiv_data_210930-054931.csv', engine='python', on_bad_lines='skip')
print(d)

                                        terms  \
0                                   ['cs.LG']   
1                          ['cs.LG', 'cs.AI']   
2               ['cs.LG', 'cs.CR', 'stat.ML']   
3                          ['cs.LG', 'cs.CR']   
4                                   ['cs.LG']   
...                                       ...   
4025                     ['cs.LG', 'stat.ML']   
4026         ['stat.ML', 'cs.LG', 'quant-ph']   
4027           ['cs.CV', 'cs.AI', 'q-bio.NC']   
4028            ['cs.LG', 'cs.CV', 'stat.ML']   
4029  ['cs.LG', 'physics.data-an', 'stat.ML']   

                                                 titles  \
0     Multi-Level Attention Pooling for Graph Neural...   
1     Decision Forests vs. Deep Networks: Conceptual...   
2     Power up! Robust Graph Convolutional Network v...   
3     Releasing Graph Neural Networks with Different...   
4     Recurrence-Aware Long-Term Cognitive Network f...   
...                                                 ...  

In [14]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
lemmatizer = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))

def preprocess_text(text):
    # Ensure text is a string to prevent errors with NaN or other types
    if pd.isna(text):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers (keeping only alphabets)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # Remove stopwords and perform lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords_set]
    # Rejoin words into a string
    return ' '.join(processed_words)

# Apply the preprocessing function to the 'titles' column (assuming 'titles' is the intended column)
d['Processed_News'] = d['titles'].apply(preprocess_text)

# Display the original and processed news
print("Original Titles:")
print(d['titles'].head())
print("\nProcessed Titles (now in 'Processed_News' column):")
print(d['Processed_News'].head())

Original Titles:
0    Multi-Level Attention Pooling for Graph Neural...
1    Decision Forests vs. Deep Networks: Conceptual...
2    Power up! Robust Graph Convolutional Network v...
3    Releasing Graph Neural Networks with Different...
4    Recurrence-Aware Long-Term Cognitive Network f...
Name: titles, dtype: object

Processed Titles (now in 'Processed_News' column):
0    multi level attention pooling graph neural net...
1    decision forest v deep network conceptual simi...
2    power robust graph convolutional network via g...
3    releasing graph neural network differential pr...
4    recurrence aware long term cognitive network e...
Name: Processed_News, dtype: object


In [16]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Re-create CountVectorizer and bow_matrix to ensure availability
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Number of topics to extract
num_topics = 2

# Initialize LDA model
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# Fit LDA model to the Bag of Words matrix
lda_model.fit(bow_matrix)

# Display the topics and their top keywords
print("Topics and their top keywords:")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_): # Corrected from components__ to components_
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model, vectorizer.get_feature_names_out(), 5)

Topics and their top keywords:
Topic 1:
language image visual learning video
Topic 2:
learning network time series neural


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'd' DataFrame with 'Processed_News' column is already available

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix (DataFrame):")
display(bow_df)

Bag of Words Matrix (DataFrame):


,aa,aamdrl,abandoning,abba,abc,abdominal,ability,abnormal,abnormality,absent,...,xxii,yasenn,year,yet,yield,yor,youmakeup,youtube,zero,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Processed_News' column
bow_matrix = vectorizer.fit_transform(d['Processed_News'])

# Convert the BoW matrix to a DataFrame for better readability
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print("Bag of Words Matrix:")
display(bow_df)

print("\nVocabulary (Feature Names):")
print(vectorizer.get_feature_names_out())

Bag of Words Matrix:


,aa,aamdrl,abandoning,abba,abc,abdominal,ability,abnormal,abnormality,absent,...,xxii,yasenn,year,yet,yield,yor,youmakeup,youtube,zero,zigzag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



Vocabulary (Feature Names):
['aa' 'aamdrl' 'abandoning' ... 'youtube' 'zero' 'zigzag']


In [19]:
from sklearn.decomposition import NMF

# Initialize NMF model
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the Bag of Words matrix
nmf_model.fit(bow_matrix)

print("NMF model fitted successfully.")

NMF model fitted successfully.


In [20]:
# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Function to display top words for each topic
def display_topics(model, feature_names, top_n=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([
            feature_names[i]
            for i in topic.argsort()[:-top_n - 1:-1]
        ]))

# Call the function
display_topics(nmf_model, feature_names, top_n=10)


Topic 1:
learning time series deep data model forecasting using based machine

Topic 2:
network neural graph convolutional using deep based recurrent attention classification


In [21]:
topic_distribution = nmf_model.transform(bow_matrix)

d['dominant_topic'] = topic_distribution.argmax(axis=1)

d[['Processed_News', 'dominant_topic']].head()

,Processed_News,dominant_topic
0,multi level attention pooling graph neural net...,1
1,decision forest v deep network conceptual simi...,1
2,power robust graph convolutional network via g...,1
3,releasing graph neural network differential pr...,1
4,recurrence aware long term cognitive network e...,1


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Processed_News' column to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(d['Processed_News'])

# Convert the TF-IDF matrix to a DataFrame for better readability (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF Matrix (first 5 rows and columns):")
print(tfidf_df.head())

print("\nVocabulary (Feature Names) from TF-IDF:")
print(tfidf_vectorizer.get_feature_names_out())

TF-IDF Matrix (first 5 rows and columns):
    aa  aamdrl  abandoning  abba  abc  abdominal  ability  abnormal  \
0  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
1  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
2  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
3  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
4  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   

   abnormality  absent  ...  xxii  yasenn  year  yet  yield  yor  youmakeup  \
0          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
1          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
2          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
3          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
4          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   

   youtube  zero  zigzag  
0      0.0   0.0     0.0  
1 

In [26]:
from sklearn.decomposition import NMF

# Initialize NMF model
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the Bag of Words matrix
nmf_model.fit(bow_matrix)

print("NMF model fitted successfully.")

NMF model fitted successfully.


In [25]:
# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Function to display top words for each topic
def display_topics(model, feature_names, top_n=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([
            feature_names[i]
            for i in topic.argsort()[:-top_n - 1:-1]
        ]))

# Call the function
display_topics(nmf_model, feature_names, top_n=10)


Topic 1:
learning time series deep data model forecasting using based machine

Topic 2:
network neural graph convolutional using deep based recurrent attention classification


In [27]:
topic_distribution = nmf_model.transform(bow_matrix)

d['dominant_topic'] = topic_distribution.argmax(axis=1)

d[['Processed_News', 'dominant_topic']].head()

,Processed_News,dominant_topic
0,multi level attention pooling graph neural net...,1
1,decision forest v deep network conceptual simi...,1
2,power robust graph convolutional network via g...,1
3,releasing graph neural network differential pr...,1
4,recurrence aware long term cognitive network e...,1


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Processed_News' column to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(d['Processed_News'])

# Convert the TF-IDF matrix to a DataFrame for better readability (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF Matrix (first 5 rows and columns):")
print(tfidf_df.head())

print("\nVocabulary (Feature Names) from TF-IDF:")
print(tfidf_vectorizer.get_feature_names_out())

TF-IDF Matrix (first 5 rows and columns):
    aa  aamdrl  abandoning  abba  abc  abdominal  ability  abnormal  \
0  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
1  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
2  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
3  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   
4  0.0     0.0         0.0   0.0  0.0        0.0      0.0       0.0   

   abnormality  absent  ...  xxii  yasenn  year  yet  yield  yor  youmakeup  \
0          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
1          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
2          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
3          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   
4          0.0     0.0  ...   0.0     0.0   0.0  0.0    0.0  0.0        0.0   

   youtube  zero  zigzag  
0      0.0   0.0     0.0  
1 

In [29]:
from sklearn.decomposition import NMF


nmf_model_tfidf = NMF(n_components=num_topics, random_state=42)


nmf_model_tfidf.fit(tfidf_matrix)

print("NMF model fitted successfully to TF-IDF data.")

NMF model fitted successfully to TF-IDF data.


In [30]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))


display_topics(nmf_model_tfidf, tfidf_feature_names, 10)


Topic 1:
time series learning forecasting deep data model classification multivariate using

Topic 2:
network neural graph convolutional deep recurrent attention learning based using


In [31]:
topic_distribution_nmf = nmf_model_tfidf.transform(tfidf_matrix)
d['dominant_topic_nmf'] = topic_distribution_nmf.argmax(axis=1) + 1 # +1 to make topic numbers 1-based

print("Dominant topics assigned to 'd' DataFrame.")
print(d[['Processed_News', 'dominant_topic_nmf']].head())

Dominant topics assigned to 'd' DataFrame.
                                      Processed_News  dominant_topic_nmf
0  multi level attention pooling graph neural net...                   2
1  decision forest v deep network conceptual simi...                   2
2  power robust graph convolutional network via g...                   2
3  releasing graph neural network differential pr...                   2
4  recurrence aware long term cognitive network e...                   2


In [32]:
from sklearn.decomposition import NMF

# Initialize NMF model
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit NMF model to the Bag of Words matrix
nmf_model.fit(bow_matrix)

print("NMF model fitted successfully.")

NMF model fitted successfully.


In [33]:
print("Topics and their top keywords for NMF:")
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model, vectorizer.get_feature_names_out(), 5)

Topics and their top keywords for NMF:
Topic 1:
learning time series deep data
Topic 2:
network neural graph convolutional using


In [34]:
print("TF-IDF Matrix (first 5 rows):\n")
display(tfidf_df.head())

print(f"\nShape of TF-IDF matrix: {tfidf_matrix.shape}")

TF-IDF Matrix (first 5 rows):



,aa,aamdrl,abandoning,abba,abc,abdominal,ability,abnormal,abnormality,absent,...,xxii,yasenn,year,yet,yield,yor,youmakeup,youtube,zero,zigzag
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Shape of TF-IDF matrix: (4030, 4402)


In [35]:
print("NMF Topics and their Top Keywords (TF-IDF based):")
display_topics(nmf_model_tfidf, tfidf_feature_names, num_top_words=10)

NMF Topics and their Top Keywords (TF-IDF based):
Topic 1:
time series learning forecasting deep data model classification multivariate using
Topic 2:
network neural graph convolutional deep recurrent attention learning based using


In [36]:
print("Dominant NMF topic assigned to each document:")
print(d[['Processed_News', 'dominant_topic_nmf']].head())

Dominant NMF topic assigned to each document:
                                      Processed_News  dominant_topic_nmf
0  multi level attention pooling graph neural net...                   2
1  decision forest v deep network conceptual simi...                   2
2  power robust graph convolutional network via g...                   2
3  releasing graph neural network differential pr...                   2
4  recurrence aware long term cognitive network e...                   2
